In [6]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("C:/Users/Pream/OneDrive/Desktop/projects/CivilBackend/augmented_construction_data_500.xlsx")

# Display the first few rows
df.head()


,Project Name,Start Date,Present Date,Days Duration,Quoted Budget,Labor Cost,Material Cost,Machinery Cost
0,Highway Masterplan,2022-05-16,2024-03-11,665,3799850,905198,1826106,552284
1,Commercial Development,2021-03-03,2024-03-11,1104,906890,203312,297258,197738
2,Bridge Overhaul,2021-10-15,2024-03-11,878,2644744,856029,1183377,432220
3,Industrial Restoration,2020-10-07,2024-03-11,1251,2814138,454502,1045705,376639
4,Smart City Urban Planning,2020-05-27,2024-03-11,1384,3398536,818076,1207811,409535


In [7]:
df["Start Date"] = pd.to_datetime(df["Start Date"], errors="coerce")
df["Present Date"] = pd.to_datetime(df["Present Date"], errors="coerce")

# Check if dates are properly converted
df.dtypes


Project Name              object
Start Date        datetime64[ns]
Present Date      datetime64[ns]
Days Duration              int64
Quoted Budget              int64
Labor Cost                 int64
Material Cost              int64
Machinery Cost             int64
dtype: object

In [8]:
# Compute Total Expenses
df["Total Expenses"] = df["Labor Cost"] + df["Material Cost"] + df["Machinery Cost"]

# Compute Cost Per Day
df["Days Passed"] = (df["Present Date"] - df["Start Date"]).dt.days.replace(0, 1)  # Prevent division by zero
df["Cost Per Day"] = df["Total Expenses"] / df["Days Passed"]

# Replace NaN or infinite values with 0
df["Cost Per Day"] = df["Cost Per Day"].fillna(0).replace([float("inf"), -float("inf")], 0)

# Display dataset
df.head()


,Project Name,Start Date,Present Date,Days Duration,Quoted Budget,Labor Cost,Material Cost,Machinery Cost,Total Expenses,Days Passed,Cost Per Day
0,Highway Masterplan,2022-05-16,2024-03-11,665,3799850,905198,1826106,552284,3283588,665,4937.726316
1,Commercial Development,2021-03-03,2024-03-11,1104,906890,203312,297258,197738,698308,1104,632.525362
2,Bridge Overhaul,2021-10-15,2024-03-11,878,2644744,856029,1183377,432220,2471626,878,2815.063781
3,Industrial Restoration,2020-10-07,2024-03-11,1251,2814138,454502,1045705,376639,1876846,1251,1500.276579
4,Smart City Urban Planning,2020-05-27,2024-03-11,1384,3398536,818076,1207811,409535,2435422,1384,1759.697977


In [9]:
# Calculate Project Progress (%)
df["Project Progress (%)"] = (df["Days Passed"] / df["Days Duration"]) * 100

# Fill negative or NaN values with 0 (for projects that haven't started yet)
df["Project Progress (%)"] = df["Project Progress (%)"].fillna(0).clip(0, 100)

# Display updated dataset
df.head()


,Project Name,Start Date,Present Date,Days Duration,Quoted Budget,Labor Cost,Material Cost,Machinery Cost,Total Expenses,Days Passed,Cost Per Day,Project Progress (%)
0,Highway Masterplan,2022-05-16,2024-03-11,665,3799850,905198,1826106,552284,3283588,665,4937.726316,100.0
1,Commercial Development,2021-03-03,2024-03-11,1104,906890,203312,297258,197738,698308,1104,632.525362,100.0
2,Bridge Overhaul,2021-10-15,2024-03-11,878,2644744,856029,1183377,432220,2471626,878,2815.063781,100.0
3,Industrial Restoration,2020-10-07,2024-03-11,1251,2814138,454502,1045705,376639,1876846,1251,1500.276579,100.0
4,Smart City Urban Planning,2020-05-27,2024-03-11,1384,3398536,818076,1207811,409535,2435422,1384,1759.697977,100.0


In [10]:
# Estimate Final Cost
df["Estimated Final Cost"] = df["Cost Per Day"] * df["Days Duration"]

# Fill missing values with the quoted budget if no progress is recorded
df["Estimated Final Cost"] = df["Estimated Final Cost"].fillna(df["Quoted Budget"])

# Display final dataset
df.head()

,Project Name,Start Date,Present Date,Days Duration,Quoted Budget,Labor Cost,Material Cost,Machinery Cost,Total Expenses,Days Passed,Cost Per Day,Project Progress (%),Estimated Final Cost
0,Highway Masterplan,2022-05-16,2024-03-11,665,3799850,905198,1826106,552284,3283588,665,4937.726316,100.0,3283588.0
1,Commercial Development,2021-03-03,2024-03-11,1104,906890,203312,297258,197738,698308,1104,632.525362,100.0,698308.0
2,Bridge Overhaul,2021-10-15,2024-03-11,878,2644744,856029,1183377,432220,2471626,878,2815.063781,100.0,2471626.0
3,Industrial Restoration,2020-10-07,2024-03-11,1251,2814138,454502,1045705,376639,1876846,1251,1500.276579,100.0,1876846.0
4,Smart City Urban Planning,2020-05-27,2024-03-11,1384,3398536,818076,1207811,409535,2435422,1384,1759.697977,100.0,2435422.0


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define Features (X) and Target (y)
X = df[['Quoted Budget', 'Labor Cost', 'Material Cost', 'Machinery Cost', 'Project Progress (%)', 'Cost Per Day']]
y = df['Estimated Final Cost']

# Split dataset into Training (80%) and Testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest Regressor
model = RandomForestRegressor(
    n_estimators=50,        # Reduce number of trees
    max_depth=10,           # Limit tree depth
    max_features="sqrt",    # Consider only sqrt(features) per split
    random_state=42
)
model.fit(X_train, y_train)

# Make Predictions
y_pred = model.predict(X_test)


In [12]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions on the test dataset
y_pred_test = model.predict(X_test)

# Compute evaluation metrics
mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = mse ** 0.5  # Manually compute RMSE
r2 = r2_score(y_test, y_pred_test)

# Display results
print(f"🔹 Mean Absolute Error (MAE): ₹{mae:.2f}")
print(f"🔹 Mean Squared Error (MSE): ₹{mse:.2f}")
print(f"🔹 Root Mean Squared Error (RMSE): ₹{rmse:.2f}")
print(f"🔹 R² Score: {r2:.2f}")



🔹 Mean Absolute Error (MAE): ₹80318.90
🔹 Mean Squared Error (MSE): ₹13270545866.54
🔹 Root Mean Squared Error (RMSE): ₹115197.86
🔹 R² Score: 0.99


In [13]:
train_r2 = model.score(X_train, y_train)
test_r2 = model.score(X_test, y_test)

print(f"Train R² Score: {train_r2:.2f}")
print(f"Test R² Score: {test_r2:.2f}")


Train R² Score: 1.00
Test R² Score: 0.99


In [14]:
import numpy as np
import pandas as pd

# New sample project data (Example)
new_data = np.array([[500000, 200000, 100000, 50000, 50, 10000]])  # Adjust values

# Convert to DataFrame using same column names as training
columns = ['Quoted Budget', 'Labor Cost', 'Material Cost', 'Machinery Cost', 'Project Progress (%)', 'Cost Per Day']
new_data_df = pd.DataFrame(new_data, columns=columns)

# Predict estimated final cost
predicted_cost = model.predict(new_data_df)[0]

# Display result
print(f"✅ Predicted Estimated Final Cost: ₹{predicted_cost:.2f}")


✅ Predicted Estimated Final Cost: ₹472835.08


In [15]:
test_cases = np.array([
    [500000, 200000, 100000, 50000, 100, 10000],  # Fully completed project
    [500000, 200000, 100000, 50000, 0, 10000],    # Project not started
    [1000000, 400000, 200000, 100000, 50, 20000], # Bigger project with 50% progress
])

test_df = pd.DataFrame(test_cases, columns=columns)
predictions = model.predict(test_df)

for i, pred in enumerate(predictions):
    print(f"🔹 Test Case {i+1} - Predicted Final Cost: ₹{pred:.2f}")


🔹 Test Case 1 - Predicted Final Cost: ₹472835.08
🔹 Test Case 2 - Predicted Final Cost: ₹472835.08
🔹 Test Case 3 - Predicted Final Cost: ₹783067.71


In [16]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model, X, y, cv=5, scoring="r2")  # 5-fold cross-validation
print(f"🔹 Cross-Validation R² Scores: {cv_scores}")
print(f"🔹 Average R² Score: {np.mean(cv_scores):.2f}")


🔹 Cross-Validation R² Scores: [0.99110987 0.99239431 0.99210995 0.99285203 0.98723689]
🔹 Average R² Score: 0.99


In [17]:
importances = model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({"Feature": feature_names, "Importance": importances})
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)
print(feature_importance_df)


                Feature  Importance
0         Quoted Budget    0.395642
2         Material Cost    0.213483
1            Labor Cost    0.201788
3        Machinery Cost    0.142478
5          Cost Per Day    0.046610
4  Project Progress (%)    0.000000


In [18]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring="r2", n_jobs=-1)
grid_search.fit(X, y)

print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_


Best Parameters: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 200}


In [19]:
# Train the optimized Random Forest model
best_model = RandomForestRegressor(n_estimators=200, max_depth=10, min_samples_split=2, random_state=42)
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_best = best_model.predict(X_test)

# Evaluate performance again
mae = mean_absolute_error(y_test, y_pred_best)
mse = mean_squared_error(y_test, y_pred_best)
rmse = mse ** 0.5  # Root Mean Squared Error
r2 = r2_score(y_test, y_pred_best)

print(f"🔹 Mean Absolute Error (MAE): ₹{mae:.2f}")
print(f"🔹 Mean Squared Error (MSE): ₹{mse:.2f}")
print(f"🔹 Root Mean Squared Error (RMSE): ₹{rmse:.2f}")
print(f"🔹 R² Score: {r2:.2f}")  # Should be close to 1 for a well-tuned model


🔹 Mean Absolute Error (MAE): ₹74861.60
🔹 Mean Squared Error (MSE): ₹11902794268.66
🔹 Root Mean Squared Error (RMSE): ₹109099.93
🔹 R² Score: 0.99


In [20]:
import numpy as np
import pandas as pd

# New sample project data (Example)
new_data = np.array([[500000, 200000, 100000, 50000, 50, 10000]])  # Adjust values

# Convert to DataFrame using same column names as training
columns = ['Quoted Budget', 'Labor Cost', 'Material Cost', 'Machinery Cost', 'Project Progress (%)', 'Cost Per Day']
new_data_df = pd.DataFrame(new_data, columns=columns)

# Predict estimated final cost
predicted_cost = model.predict(new_data_df)[0]

# Display result
print(f"✅ Predicted Estimated Final Cost: ₹{predicted_cost:.2f}")


✅ Predicted Estimated Final Cost: ₹472835.08


In [21]:
import joblib

# Save the trained model to a file
joblib.dump(model, "final_cost_prediction_model.pkl")

print("✅ Model saved successfully!")


✅ Model saved successfully!
